#2 한 번 처리한 변수들에 RFECV

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import pandas_profiling

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import LabelEncoder

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

import xgboost as xgb

from datetime import datetime
import lightgbm as lgb
#import multiprocessing
#import gc

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
import os
import gc
import sys
sys.path.append("../..")

In [5]:
os.getcwd()

'/home/biglake0987/ybigta_first_proj/EDA'

In [6]:
os.listdir()

['FinalPT_RFECV---1.ipynb',
 'FinalPT_RFECV-1_XGBoost(error).ipynb',
 'test_transaction.csv',
 'rfecv_1_train.csv',
 'V_cols_test.csv',
 'train_transaction.csv',
 'V_cols_train.csv',
 'FinalPT_RFECV-----1.ipynb',
 '.ipynb_checkpoints',
 'FinalPT_RFECV-1.ipynb',
 'rfecv_1_test.csv',
 'jinoo_train.csv',
 'FinalPT_RFECV-1_XGBoost(x).ipynb',
 'FinalPT_RFECV-----2.ipynb',
 'test_identity.csv',
 'train_identity.csv',
 'FinalPT_RFECV-----1(to_csv).ipynb',
 'sample_submission.csv',
 'sample.ipynb',
 'to_csv(진우).ipynb',
 'mintae1.ipynb',
 'jinoo_test.csv',
 'V_cols(20190814).ipynb',
 '.gitkeep']

In [149]:
trans = pd.read_csv('train_transaction.csv', index_col='TransactionID')
test_trans = pd.read_csv('test_transaction.csv', index_col='TransactionID')

In [150]:
ident = pd.read_csv('train_identity.csv', index_col='TransactionID')
test_ident = pd.read_csv('test_identity.csv', index_col='TransactionID')

In [151]:
sample_sub = pd.read_csv('sample_submission.csv', index_col='TransactionID')

In [152]:
train = trans.merge(ident, how='left', left_index=True, right_index=True)
test = test_trans.merge(test_ident, how='left', left_index=True, right_index=True)

In [153]:
print(train.shape)
print(test.shape)

(590540, 433)
(506691, 432)


In [154]:
del trans, ident, test_trans, test_ident
gc.collect()

3476

for column in ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2']:
    train[column +  '_count_full'] = \
        train[column].map(pd.concat([train[column], test[column]], ignore_index=True).value_counts(dropna=False))
    test[column +  '_count_full'] = \
        test[column].map(pd.concat([train[column], test[column]], ignore_index=True).value_counts(dropna=False))

train['Transaction_day_of_week'] = np.floor((train['TransactionDT'] / (3600 * 24) - 1) % 7)
test['Transaction_day_of_week'] = np.floor((test['TransactionDT'] / (3600 * 24) - 1) % 7)

train['Transaction_hour_of_day'] = np.floor(train['TransactionDT'] / 3600) % 24
test['Transaction_hour_of_day'] = np.floor(test['TransactionDT'] / 3600) % 24


train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')

train[['R_emaildomain', 'P_emaildomain']].apply(lambda x: x[0] if x[0] == np.nan else x[1], axis=1).head()

train['emaildomain'] = train[['R_emaildomain', 'P_emaildomain']].apply(lambda x: x[0] if x[0] == np.nan else x[1], axis=1)
test['emaildomain'] = test[['R_emaildomain', 'P_emaildomain']].apply(lambda x: x[0] if x[0] == np.nan else x[1], axis=1)

train = train.drop(columns=['R_emaildomain', 'P_emaildomain'])
test = test.drop(columns=['R_emaildomain', 'P_emaildomain'])

In [155]:
train.columns

Index(['isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1',
       'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=433)

In [156]:
def print_columns(df):
    cols = df.columns
    string = ''
    for col in cols:
        string += '"%s", ' % str(col)
    print(string)
    print(len(string))

In [157]:
print_columns(train)

"isFraud", "TransactionDT", "TransactionAmt", "ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", "addr1", "addr2", "dist1", "dist2", "P_emaildomain", "R_emaildomain", "C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", "C9", "C10", "C11", "C12", "C13", "C14", "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12", "D13", "D14", "D15", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "V10", "V11", "V12", "V13", "V14", "V15", "V16", "V17", "V18", "V19", "V20", "V21", "V22", "V23", "V24", "V25", "V26", "V27", "V28", "V29", "V30", "V31", "V32", "V33", "V34", "V35", "V36", "V37", "V38", "V39", "V40", "V41", "V42", "V43", "V44", "V45", "V46", "V47", "V48", "V49", "V50", "V51", "V52", "V53", "V54", "V55", "V56", "V57", "V58", "V59", "V60", "V61", "V62", "V63", "V64", "V65", "V66", "V67", "V68", "V69", "V70", "V71", "V72", "V73", "V74", "V75", "V76", "V77", "V78", "V79", "V80", "V81", "V82", "V83", "V84",

# 준형이형꺼 처리
#id_30

def os(df):
    if df == "-999":
        df = -999
    elif "Windows" in df:
        df = "Windows"
    elif "Mac" in df:
        df = "Mac"
    elif "iOS" in df:
        df = "ios"
    elif "Android" in df:
        df = "Android"
    else:
        df = "other"
        
    return df

train["id_30"].fillna("-999", inplace=True)
train["OS"] = train["id_30"].apply(os)

test["id_30"].fillna("-999", inplace=True)
test["OS"] = test["id_30"].apply(os)

#id_31 
# latest 여부 / 브라우저 col 생성
latest_browser_list = ["samsung browser 7.0",
                  "opera 53.0",
                  "mobile safari 10.0",
                  "google search application 49.0",
                  "firefox 60.0",
                  "edge 17.0",
                  "chrome 69.0", 
                  "chrome 67.0 for android",
                  "chrome 63.0 for android",
                  "chrome 63.0 for ios",
                  "chrome 64.0",
                  "chrome 64.0 for android",
                  "chrome 64.0 for ios",
                  "chrome 65.0",
                  "chrome 65.0 for android",
                  "chrome 65.0 for ios",
                  "chrome 66.0",
                  "chrome 66.0 for android",
                  "chrome 66.0 for ios"
                 ]

def latest_browser(df):
    if df == "-999":
        df = -999
    elif df in latest_browser_list:
        df = 1
    else:
        df = 0
    return df

def browser(df):
    if df == "-999":
        df = -999
    elif "chrome" in df:
        df = "chrome"
    elif "safari" in df:
        df = "safari"
    elif "firefox" in df:
        df = "firefox"
    elif "ie" in df:
        df = "ie"
    elif "samsung" in df:
        df = "samsumg"
    elif "Samsung" in df:
        df = "samsumg"
    return df 
    

train["id_31"].fillna("-999", inplace=True)
train["Latest_browser"] = train["id_31"].apply(latest_browser)
train["Browser"] = train["id_31"].apply(browser)

test["id_31"].fillna("-999", inplace=True)
test["Latest_browser"] = test["id_31"].apply(latest_browser)
test["Browser"] = test["id_31"].apply(browser)

In [158]:
train.columns

Index(['isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1',
       'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=433)

In [159]:
print_columns(train)

"isFraud", "TransactionDT", "TransactionAmt", "ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", "addr1", "addr2", "dist1", "dist2", "P_emaildomain", "R_emaildomain", "C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", "C9", "C10", "C11", "C12", "C13", "C14", "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12", "D13", "D14", "D15", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "V10", "V11", "V12", "V13", "V14", "V15", "V16", "V17", "V18", "V19", "V20", "V21", "V22", "V23", "V24", "V25", "V26", "V27", "V28", "V29", "V30", "V31", "V32", "V33", "V34", "V35", "V36", "V37", "V38", "V39", "V40", "V41", "V42", "V43", "V44", "V45", "V46", "V47", "V48", "V49", "V50", "V51", "V52", "V53", "V54", "V55", "V56", "V57", "V58", "V59", "V60", "V61", "V62", "V63", "V64", "V65", "V66", "V67", "V68", "V69", "V70", "V71", "V72", "V73", "V74", "V75", "V76", "V77", "V78", "V79", "V80", "V81", "V82", "V83", "V84",

In [160]:
train['TransactionAmt'] = np.log(train['TransactionAmt']+1)
test['TransactionAmt'] = np.log(test['TransactionAmt']+1)

final_cols = ['isFraud','TransactionAmt', "ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", 
"addr1", "addr2", "dist1", "dist2", "C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", "C9", "C10", 
"C11", "C12", "C13", "C14", "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12", 
"D13", "D14", "D15", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", 'card1_count_full', 
'card2_count_full', 'card3_count_full', 'card4_count_full', 
'card5_count_full', 'card6_count_full', 'addr1_count_full', 'addr2_count_full', 'Transaction_day_of_week', 
'Transaction_hour_of_day', 'TransactionAmt_to_mean_card1', 'TransactionAmt_to_mean_card4',
'TransactionAmt_to_std_card1', 'TransactionAmt_to_std_card4', 'emaildomain',"id_01", "id_02", 
"id_03", "id_04", "id_05", "id_06", "id_07", "id_08", "id_09", "id_10", "id_11", 
"id_12", "id_13", "id_14", "id_15", "id_16", "id_17", "id_18", "id_19", "id_20", "id_21", "id_22", "id_23", "id_24", 
"id_25", "id_26", "id_27", "id_28", "id_29", "id_30", "id_31", "id_32", "id_33", "id_34", "id_35", 
"id_36", "id_37", "id_38", "OS", "Latest_browser", "Browser",
'V1','V2','V3','V4','V5','V6','V7','V9','V12','V13','V17','V18','V19',
'V20','V23','V24','V25','V26','V29','V30','V33',
 'V34','V35','V36','V37','V38','V39','V40','V42','V43','V44','V45','V47','V48','V49','V51','V52','V53','V54','V55',
 'V56','V57','V58','V59','V60','V61','V62','V63','V64','V66','V67','V69','V70','V71','V72','V73','V74','V75','V76',
 'V77','V78','V79','V80','V81','V82','V83','V85','V86','V87','V90','V91','V92','V93','V94', 'V95', 'V96', 'V97',
 'V98','V99','V100','V101','V102','V103','V105','V109','V115','V124','V125','V126','V127','V128','V129', 'V130','V131',
 'V132', 'V133', 'V134','V135','V136','V137','V139','V140','V143','V145','V146','V147','V149','V150','V151','V152',
 'V156','V158','V159','V160','V161','V162','V164','V165','V166',
 'V167','V168','V169','V170','V171','V172','V173','V174','V175','V176','V177','V178','V179','V180','V182','V187','V188',
 'V189','V192','V198','V199','V200','V201','V202','V203','V204','V205','V206','V207','V208','V209','V210',
 'V211', 'V212','V213','V214','V215','V216','V217','V218','V219','V220','V221','V222','V223','V224','V225','V226',
 'V227','V228','V229','V230','V231','V232','V233','V234','V236','V239','V243','V244','V245','V246','V248','V250',
 'V251','V256','V257',
 'V258','V259','V260','V261','V262','V263','V264','V265','V266','V267','V268','V270','V271','V272','V273','V274',
 'V275','V276','V277','V278','V279','V280','V281','V282','V283','V284','V285','V286','V287','V288','V289','V290',
 'V291','V292','V293','V294','V295',
 'V296','V297','V298','V299','V300','V301','V303','V304','V306','V307','V308','V309','V310','V311','V312',
 'V313','V314','V315','V316','V317','V318','V319','V320','V321','V322','V323','V324','V326','V329','V331',
 'V332','V333','V335','V336','V338','V339']

In [179]:
final_cols = ['isFraud','TransactionAmt', "ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", 
"addr1", "addr2", "dist1", "dist2", "C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", "C9", "C10", 
"C11", "C12", "C13", "C14", "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12", 
"D13", "D14", "D15", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "id_01", "id_02", 
"id_03", "id_04", "id_05", "id_06", "id_07", "id_08", "id_09", "id_10", "id_11", 
"id_12", "id_13", "id_14", "id_15", "id_16", "id_17", "id_18", "id_19", "id_20", "id_21", "id_22", "id_23", "id_24", 
"id_25", "id_26", "id_27", "id_28", "id_29", "id_30", "id_31", "id_32", "id_33", "id_34", "id_35", 
"id_36", "id_37", "id_38", 
'V1','V2','V3','V4','V5','V6','V7','V9','V12','V13','V17','V18','V19',
'V20','V23','V24','V25','V26','V29','V30','V33',
 'V34','V35','V36','V37','V38','V39','V40','V42','V43','V44','V45','V47','V48','V49','V51','V52','V53','V54','V55',
 'V56','V57','V58','V59','V60','V61','V62','V63','V64','V66','V67','V69','V70','V71','V72','V73','V74','V75','V76',
 'V77','V78','V79','V80','V81','V82','V83','V85','V86','V87','V90','V91','V92','V93','V94', 'V95', 'V96', 'V97',
 'V98','V99','V100','V101','V102','V103','V105','V109','V115','V124','V125','V126','V127','V128','V129', 'V130','V131',
 'V132', 'V133', 'V134','V135','V136','V137','V139','V140','V143','V145','V146','V147','V149','V150','V151','V152',
 'V156','V158','V159','V160','V161','V162','V164','V165','V166',
 'V167','V168','V169','V170','V171','V172','V173','V174','V175','V176','V177','V178','V179','V180','V182','V187','V188',
 'V189','V192','V198','V199','V200','V201','V202','V203','V204','V205','V206','V207','V208','V209','V210',
 'V211', 'V212','V213','V214','V215','V216','V217','V218','V219','V220','V221','V222','V223','V224','V225','V226',
 'V227','V228','V229','V230','V231','V232','V233','V234','V236','V239','V243','V244','V245','V246','V248','V250',
 'V251','V256','V257',
 'V258','V259','V260','V261','V262','V263','V264','V265','V266','V267','V268','V270','V271','V272','V273','V274',
 'V275','V276','V277','V278','V279','V280','V281','V282','V283','V284','V285','V286','V287','V288','V289','V290',
 'V291','V292','V293','V294','V295',
 'V296','V297','V298','V299','V300','V301','V303','V304','V306','V307','V308','V309','V310','V311','V312',
 'V313','V314','V315','V316','V317','V318','V319','V320','V321','V322','V323','V324','V326','V329','V331',
 'V332','V333','V335','V336','V338','V339']

train['card1_count_full'] = np.log(train['card1_count_full']+1)
test['card1_count_full'] = np.log(test['card1_count_full']+1)

train['card2_count_full'] = np.log(train['card2_count_full']+1)
test['card2_count_full'] = np.log(test['card2_count_full']+1)

train['card3_count_full'] = np.log(train['card3_count_full']+1)
test['card3_count_full'] = np.log(test['card3_count_full']+1)

train['card4_count_full'] = np.log(train['card4_count_full']+1)
test['card4_count_full'] = np.log(test['card4_count_full']+1)

train['card5_count_full'] = np.log(train['card5_count_full']+1)
test['card5_count_full'] = np.log(test['card5_count_full']+1)

train['card6_count_full'] = np.log(train['card6_count_full']+1)
test['card6_count_full'] = np.log(test['card6_count_full']+1)

train['addr1_count_full'] = np.log(train['addr1_count_full']+1)
test['addr1_count_full'] = np.log(test['addr1_count_full']+1)

train['addr2_count_full'] = np.log(train['addr2_count_full']+1)
test['addr2_count_full'] = np.log(test['addr2_count_full']+1)

j_train = pd.read_csv('jinoo_train.csv')

In [129]:
j_train.columns

Index(['card1_count_full', 'card2_count_full', 'card3_count_full',
       'card4_count_full', 'card5_count_full', 'card6_count_full',
       'addr1_count_full', 'addr2_count_full', 'Transaction_day_of_week',
       'Transaction_hour_of_day', 'TransactionAmt_to_mean_card1',
       'TransactionAmt_to_mean_card4', 'TransactionAmt_to_std_card1',
       'TransactionAmt_to_std_card4', 'emaildomain'],
      dtype='object')

tttrain = pd.read_csv('train_transaction.csv', index_col='TransactionID')


def check_and_drop_column(df, column):
    if column in df.columns:
        df.drop(columns=[column], axis=1, inplace=True)

 for col in tttrain.columns:
    if str(col)[0] == 'V':
        check_and_drop_column(tttrain, col)

tttrain.columns

In [210]:

del_cols2 = []
no_cols2= []
for col in train.columns:

  if col not in final_cols:
    del_cols2.append(col)
  else:
    no_cols2.append(col)
print(len(no_cols2));print(len(del_cols2))
print(no_cols2 == final_cols)

348
85
False


In [213]:
del_cols_test2 = del_cols2
len(del_cols_test2)

85

In [214]:
print(len(del_cols2))
del_cols

85


['TransactionDT',
 'P_emaildomain',
 'R_emaildomain',
 'V8',
 'V10',
 'V11',
 'V14',
 'V15',
 'V16',
 'V21',
 'V22',
 'V27',
 'V28',
 'V31',
 'V32',
 'V41',
 'V46',
 'V50',
 'V65',
 'V68',
 'V84',
 'V88',
 'V89',
 'V104',
 'V106',
 'V107',
 'V108',
 'V110',
 'V111',
 'V112',
 'V113',
 'V114',
 'V116',
 'V117',
 'V118',
 'V119',
 'V120',
 'V121',
 'V122',
 'V123',
 'V138',
 'V141',
 'V142',
 'V144',
 'V148',
 'V153',
 'V154',
 'V155',
 'V157',
 'V163',
 'V181',
 'V183',
 'V184',
 'V185',
 'V186',
 'V190',
 'V191',
 'V193',
 'V194',
 'V195',
 'V196',
 'V197',
 'V235',
 'V237',
 'V238',
 'V240',
 'V241',
 'V242',
 'V247',
 'V249',
 'V252',
 'V253',
 'V254',
 'V255',
 'V269',
 'V302',
 'V305',
 'V325',
 'V327',
 'V328',
 'V330',
 'V334',
 'V337',
 'DeviceType',
 'DeviceInfo',
 'isFraud',
 'C3',
 'M1',
 'id_04',
 'id_07',
 'id_08',
 'id_10',
 'id_11',
 'id_12',
 'id_21',
 'id_22',
 'id_23',
 'id_24',
 'id_25',
 'id_26',
 'id_27',
 'id_28',
 'id_29',
 'id_34',
 'id_35',
 'id_37',
 'V1',
 'V2

In [183]:
one_train = train
two_train = train
three_train = train
four_train = train
five_train = train

In [184]:
one_test = test
two_test = test
three_test = test
four_test = test
five_test = test

In [185]:
from tqdm import tqdm_notebook

In [186]:
for col in tqdm_notebook(one_train.columns): 
    if one_train[col].dtype == 'object':
        le = LabelEncoder()
        le.fit(list(one_train[col].astype(str).values) + list(one_test[col].astype(str).values))
        one_train[col] = le.transform(list(one_train[col].astype(str).values))

In [169]:
one_train = reduce_mem_usage(one_train)
one_test = reduce_mem_usage(one_test)



Mem. usage decreased to 547.70 Mb (72.3% reduction)
Mem. usage decreased to 583.43 Mb (65.6% reduction)


In [188]:
one_train = one_train.drop(del_cols, axis=1)
one_test = one_test.drop(del_cols, axis=1)

In [215]:
three_train = three_train.drop(del_cols2, axis=1)
three_test = three_test.drop(del_cols2, axis=1)

In [189]:
one_train.columns

Index(['isFraud', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3',
       'card4', 'card5', 'card6', 'addr1',
       ...
       'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38'],
      dtype='object', length=348)

In [190]:
one_test.columns

Index(['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4',
       'card5', 'card6', 'addr1', 'addr2',
       ...
       'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38'],
      dtype='object', length=347)

for col in one_train.columns:
    if col != 'isFraud':
        one_train[col] = np.log(one_train[col]+1)
    else:
        print("isFraud pass")
        
for col in one_test.columns:
    one_test[col] = np.log(one_test[col]+1)

In [191]:
one_train.describe()

,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
count,590540.000000,590540.000000,590540.000000,590540.000000,581607.0,588975.0,590540.000000,586281.0,590540.000000,524834.0,...,590540.000000,590540.000000,590540.000000,77586.0,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000
mean,0.034990,NaN,3.220757,9898.734658,NaN,NaN,3.266305,NaN,1.753033,NaN,...,1.634396,80.935500,123.923744,NaN,426.594436,3.837403,1.654289,1.534238,1.709557,1.636084
std,0.183755,0.000000,1.423926,4901.170153,NaN,0.0,1.035866,0.0,0.449543,NaN,...,0.696917,14.756863,29.237451,0.0,97.397769,0.446413,0.663430,0.838376,0.556321,0.694144
min,0.000000,0.223999,0.000000,1000.000000,100.0,100.0,0.000000,100.0,0.000000,100.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,3.791016,3.000000,6019.000000,214.0,150.0,2.000000,166.0,1.000000,204.0,...,2.000000,86.000000,136.000000,24.0,461.000000,4.000000,2.000000,2.000000,2.000000,2.000000
50%,0.000000,4.246094,4.000000,9678.000000,361.0,150.0,4.000000,226.0,2.000000,299.0,...,2.000000,86.000000,136.000000,24.0,461.000000,4.000000,2.000000,2.000000,2.000000,2.000000
75%,0.000000,4.835938,4.000000,14184.000000,512.0,150.0,4.000000,226.0,2.000000,330.0,...,2.000000,86.000000,136.000000,32.0,461.000000,4.000000,2.000000,2.000000,2.000000,2.000000
max,1.000000,10.375000,4.000000,18396.000000,600.0,231.0,4.000000,237.0,4.000000,540.0,...,2.000000,87.000000,172.000000,32.0,461.000000,4.000000,2.000000,2.000000,2.000000,2.000000


## rfecv

In [192]:
X = one_train.sort_values('TransactionAmt').drop(['isFraud'], axis=1)
y = one_train.sort_values('TransactionAmt')['isFraud']

In [193]:
# RFECV does not support NaNs
X.fillna(-999, inplace=True)

In [196]:
print(one_train.shape)
print(one_test.shape)

(590540, 348)
(506691, 347)


In [197]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
         }


In [198]:
clf = lgb.LGBMClassifier(**params)
rfecv = RFECV(estimator=clf, step=10, cv=KFold(n_splits=5, shuffle=False), scoring='roc_auc', verbose=2)

In [199]:
rfecv.fit(X, y)

Fitting estimator with 347 features.
Fitting estimator with 337 features.
Fitting estimator with 327 features.
Fitting estimator with 317 features.
Fitting estimator with 307 features.
Fitting estimator with 297 features.
Fitting estimator with 287 features.
Fitting estimator with 277 features.
Fitting estimator with 267 features.
Fitting estimator with 257 features.
Fitting estimator with 247 features.
Fitting estimator with 237 features.
Fitting estimator with 227 features.
Fitting estimator with 217 features.
Fitting estimator with 207 features.
Fitting estimator with 197 features.
Fitting estimator with 187 features.
Fitting estimator with 177 features.
Fitting estimator with 167 features.
Fitting estimator with 157 features.
Fitting estimator with 147 features.
Fitting estimator with 137 features.
Fitting estimator with 127 features.
Fitting estimator with 117 features.
Fitting estimator with 107 features.
Fitting estimator with 97 features.
Fitting estimator with 87 features.
Fit

RFECV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
      estimator=LGBMClassifier(bagging_fraction=0.4181193142567742,
                               bagging_seed=11, boosting_type='gbdt',
                               class_weight=None, colsample_bytree=1.0,
                               feature_fraction=0.3797454081646243,
                               importance_type='split',
                               learning_rate=0.006883242363721497, max_depth=-1,
                               metric='auc', min_child_samples=20,
                               min_child_we....03454472573214212,
                               min_data_in_leaf=106, min_split_gain=0.0,
                               n_estimators=100, n_jobs=-1, num_leaves=491,
                               objective='binary', random_state=47,
                               reg_alpha=0.3899927210061127,
                               reg_lambda=0.6485237330340494, silent=True,
                               subsam

In [201]:
print('Optimal number of features:', rfecv.n_features_)

Optimal number of features: 267


In [200]:
all_final_cols = [col for col in X.columns[rfecv.ranking_ == 1]]
print(len(all_final_cols))
all_final_cols

267


['TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'dist2',
 'C1',
 'C2',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'V3',
 'V4',
 'V5',
 'V7',
 'V12',
 'V13',
 'V19',
 'V20',
 'V23',
 'V24',
 'V25',
 'V26',
 'V29',
 'V30',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V44',
 'V45',
 'V47',
 'V48',
 'V49',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V66',
 'V67',
 'V69',
 'V70',
 'V74',
 'V75',
 'V76',
 'V77',
 'V78',
 'V81',
 'V82',
 'V83',
 'V85',
 'V86',
 'V87',
 'V90',
 'V91',
 'V94',
 'V95',
 'V96',
 'V97',
 'V99',
 'V100',
 'V102',
 'V103',
 'V105',
 'V115',
 'V124',
 'V126',
 'V127',
 'V128',
 'V129',
 'V130',
 'V131',
 'V132',
 'V

In [206]:
all_del_cols = []
all_no_cols= []
for col in final_cols:

  if col not in all_final_cols:
    all_del_cols.append(col)
  else:
    all_no_cols.append(col)
print(len(all_no_cols));print(len(all_del_cols))
print(all_del_cols == all_final_cols)

267
81
False


In [217]:
all_del_cols.pop(0)

'isFraud'

In [218]:
all_del_cols

['C3',
 'M1',
 'id_04',
 'id_07',
 'id_08',
 'id_10',
 'id_11',
 'id_12',
 'id_21',
 'id_22',
 'id_23',
 'id_24',
 'id_25',
 'id_26',
 'id_27',
 'id_28',
 'id_29',
 'id_34',
 'id_35',
 'id_37',
 'V1',
 'V2',
 'V6',
 'V9',
 'V17',
 'V18',
 'V40',
 'V42',
 'V43',
 'V59',
 'V71',
 'V72',
 'V73',
 'V79',
 'V80',
 'V92',
 'V93',
 'V98',
 'V101',
 'V109',
 'V125',
 'V135',
 'V146',
 'V150',
 'V156',
 'V168',
 'V172',
 'V173',
 'V174',
 'V176',
 'V177',
 'V178',
 'V179',
 'V180',
 'V182',
 'V188',
 'V192',
 'V199',
 'V214',
 'V220',
 'V227',
 'V228',
 'V230',
 'V236',
 'V246',
 'V250',
 'V259',
 'V260',
 'V270',
 'V272',
 'V273',
 'V297',
 'V299',
 'V300',
 'V303',
 'V304',
 'V326',
 'V329',
 'V338',
 'V339']

In [219]:
three_train = three_train.drop(all_del_cols, axis=1)
three_test = three_test.drop(all_no_cols, axis=1)

In [220]:
train = three_train
test = three_test

In [225]:
all_no_cols

['TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'dist2',
 'C1',
 'C2',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'id_01',
 'id_02',
 'id_03',
 'id_05',
 'id_06',
 'id_09',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_30',
 'id_31',
 'id_32',
 'id_33',
 'id_36',
 'id_38',
 'V3',
 'V4',
 'V5',
 'V7',
 'V12',
 'V13',
 'V19',
 'V20',
 'V23',
 'V24',
 'V25',
 'V26',
 'V29',
 'V30',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V44',
 'V45',
 'V47',
 'V48',
 'V49',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V66',
 'V67',
 'V69',
 'V70',
 'V74',
 'V75',
 'V76',
 'V77',
 'V78',
 'V81',
 'V82',
 'V83',


In [227]:
train.to_csv('rfecv_2_train.csv')
test.to_csv('rfecv_2_test.csv')